# Memoria TFM Eduardo Hernanz - David Sánchez

El presente documento trata de guiar paso a paso al lector para poder comprender las decisiones tomadas tanto a nivel de 'negocio' como técnico

## Introducción

El proyecto consiste en el desarrollo de un simulador que permita optimizar el tiempo empleado por un triatleta para recorrer el segmento de ciclismo de un triatlon

Se ha seleccionado el segmento de ciclismo, puesto que es dónde habitualmente se desarrolla la estrategia en un Triathlon distancia Ironman (3,8km de natación, 180km de ciclismo y 42km de carrera). La mayoría de triatletas cuentan ya con potenciómetro en los relojes, que miden potencia, pulso, velocidad, altitud, tiempo, temperatura, etc

Porqué es relevante? Los triatletas de distancia Ironman entrenan durante meses para preparar esta prueba. Normalmente el éxito o fracaso de un Ironman reside en la estrategia seguida durante el ciclismo. Por tanto, con el simulador, se pretende ayudar a los competidores a mejorar sus marcas, a clasificar para el Campeonato del Mundo en Hawaii o simplemente a terminar la prueba

Actualmente no existe ningún simulador de este estilo, ni gratis ni de pago que permita desarrollar una estrategia de potencia personalizada. Existen algunas páginas que calculan datos teóricos de coeficientes aerodinámicos y de rozamiento con los que en teoría pueden calcularse velocidades teóricas, etc. Otras páginas se centran en proveer de una 'cheat sheet' de potencias basadas en datos teóricos

**_Ver el documento 'info.txt' alojado en la carpeta Memoria para saber paquetes a instalar y dependencias tecnológicas_**

Durante el Proyecto, se han desarrollado las siguientes piezas de código

1 - Scraping de las condiciones climatológicas de los entrenamientos usados para entrenar el modeloVer el notebook ../Entrenamientos/Analisis Track (Machine Learning) N_files.ipynb

>Ver el Notebook: ../Entrenamientos/Weather_History.ipynb

2 - Limpieza y creación de 'features' en los entrenamientos
Ver el Notebook: ../Entrenamientos/Analisis Track (Machine Learning) N_files.ipynb

3 - Red Neuronal para estimar la velocidad media necesaria para recorrer un tramo en función de la potencia aplicada, el desnivel y el viento aparente

>Ver más abajo el enlace al Notebook

4 - Simulador que prueba sobre cada tramo del recorrido objetivo diferentes potencias con el fin de minimizar el tiempo total cumpliendo con una potencia media objetivo

>Ver más abajo el enlace al Notebook

## La Potencia

La Potencia viene dada por la siguiente formula

$P = \frac{\rho}{2}(V+wV)^2VCdA+(Cr+\%)mgV$

Dónde:

P es la potencia necesaria

$\rho$ es la densidad del aire

V es la velocidad

wV es velocidad el viento

Cd es el coeficiente de drag

A es el área efectiva

Cr es el coeficiente de rozamiento

$\%$ es la pendiente

m es la masa total del conjunto bicicleta - ciclista

**Para comprender la importancia de los factores que intervienen podemos dividirla en la potencia necesaria para vencer la resistencia aerodinámica y la potencia para vencer factores como el rozamiento y la gravedad**

La potencia total es:

$P = \frac{\rho}{2}(V+Wv)^2VCdA+(Cr+\%)mgV$

Y se puede descomponer en la potencia necesaria para vencer al viento y para vencer el rozamiento

Potencia Aerodinámica: $Pa = \frac{\rho}{2}(V+Wv)^2VCdA$

Potencia Rozamiento: $Pr = (Cr)mgV$

Potencia Gravedad: $Pg = (\%)mgV$

In [1]:
import plotly as py
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
from IPython.core.display import HTML
import math as mt
py.offline.init_notebook_mode(connected=True)

In [2]:
def show_graph(CdA,Pendiente,Viento):

    V = np.arange(0,60,0.5)

    ro = 1.2
    #cda = 0.3
    cr = 0.0033
    m = 90
    g = 9.81
    s = mt.sin(mt.atan(Pendiente/100))

    Pa = ro/2*CdA*((V/3.6)+(Viento/3.6))**2*(V/3.6)
    Pr = (cr)*m*g*(V/3.6)
    Pg = (s)*m*g*(V/3.6)
    Pt = Pa + Pr + Pg

    trace0 = go.Scatter(
        x=V,
        y=Pa,
        mode='lines',
        name='Pot_aerodinamica'
    )

    trace1 = go.Scatter(
        x=V,
        y=Pr,
        mode='lines',
        name='Pot_rozamiento'
    )

    trace2 = go.Scatter(
        x=V,
        y=Pg,
        mode='lines',
        name='Pot_pendinte'
    )
    
    trace3 = go.Scatter(
        
        x=V,
        y=Pt,
        mode='lines',
        name='Pot_total'
    )
    
    layout = go.Layout(title='Simulador de potencia (Potencia vs. Velocidad)')
    
    data = [trace0, trace1, trace2, trace3]

    fig = dict(data=data, layout=layout)

    py.offline.iplot(fig)
widgets.interact(show_graph, CdA=(0.1,0.5,0.05), Pendiente=(-10,10,0.5), Viento=(-15,15,1))

<function __main__.show_graph>

**Primeras Conclusiones acerca de la Potencia**
* Como puede observarse en el gráfico anterior, la potencia necesaria para incrementar la velocidad depende sobre todo de la velocidad a la que estemos circulando. A más velocidad, mayor incremento de potencia para ganar 1 km/h
* Si además hay aire en contra (Viento>0), el problema se agrava _(recodar que el viento va elevado al cuadrado)_
* En cambio, con el % de pendiente, se ve que los watios 'invertidos' se transforman casi linealmente en velocidad, puesto que la parte aerodinámica tiene menos influencia al circular a baja velocidad
* Por tanto, parece que existe una oportunidad clara de repartir los watios del ciclista en los tramos más favorables, ahorrando en aquellos en los que los vamos a perder mayoritariamente luchando contra el viento o la pendiente
* Si ahora pintamos sólamente la potencia aerodinámica y vemos como varía en función de la velocidad y el viento, obtenemos lo siguiente

In [3]:
def show_graph():

    V = np.arange(30,50,1)
    wind = np.arange(-10,10,1)

    ro = 1.2
    cda = 0.25
    
    z = []

    for v in V:
        new_row = []
        for w in wind:
            new_row.append( ro/2*cda*((v/3.6)+(w/3.6))**2*(v/3.6) )
        z.append(list(new_row))

    trace0 = go.Heatmap(
        x=V,
        y=wind,
        z=z,
        colorscale='Jet'
    )

    data = [trace0]
    
    layout = go.Layout(title='Análisis de Potencia Aerodinámica (Viento(y) vs Velocidad(x))')

    fig = dict(data=data, layout=layout)

    py.offline.iplot(fig)

widgets.interact(show_graph)

<function __main__.show_graph>

**Podemos apreciar como con aire en contra, debemos reducir mucho la velocidad para mantener la potencia constante**

Una vez que hemos visto los factores que intervienen en la problemática, tenemos que tratar de obtener los valores de CdA y Cr

En el notebook "RandomForestRegressor (CdA y Cr, P en funcion V)" se trata de obtener valores de CdA y Cr. Sin embargo, los resultados no son concluyentes, debido a que las mediciones de CdA y Cr deben realizarse en condiciones controladas

>Ver el Notebook: ../Machine Learning/RandomForestRegressor (CdA y Cr, P en funcion V).ipynb

Tras los intentos de averiguar el CdA y el Cr y dado que el objetivo final es tener una alta capacidad de predicción, hemos cambiado el enfoque. Por este motivo se han probado redes neuronales que han demostrado tener un poder predictivo muy alto

>Ver el Notebook: ../RNN/TF.ipynb

En este punto, ya tenemos los datos analizados, limpios, enriquecidos y hemos desarrollado una Red Neuronal que nos da unos buenos valores predictivos

Podemos por tanto, plantearnos desarrollar un simulador que asigne potencias a cada tramo de forma que se minimice el tiempo total cumpliendo ciertas restricciones de potencia

Sin embargo antes de entrar en este punto conviene explicar el método seguido para inicializar las potencias de simulación. No es lo mismo una etapa con un puerto de 1h de duración y luego una bajada, que una en la que ese mismo desnivel se recorra en sucesivos toboganes por ser una carretera ondulada


In [4]:
def graph(pot_min, pot_max, ff_1, ff_2):
    porc = np.arange(-10,10,0.5)

    potencia = pot_min+(pot_max-pot_min)/(1+ff_1*np.e**(ff_2*porc*-1))
    potencia_min = pot_min+(pot_max-25-pot_min)/(1+ff_1*np.e**(ff_2*porc*-1))
    potencia_max = pot_min+25+(pot_max-pot_min-25)/(1+ff_1*np.e**(ff_2*porc*-1))

    trace0 = go.Scatter(        
            x=porc,
            y=potencia,
            mode='lines',
            name='Potencia_obj')

    trace1 = go.Scatter(        
            x=porc,
            y=potencia_min,
            mode='lines',
            name='Potencia_min')

    trace2 = go.Scatter(        
            x=porc,
            y=potencia_max,
            mode='lines',
            name='Potencia_max')




    layout = go.Layout(title='Sigmoide de potencia')

    data = [trace0, trace1, trace2]

    fig = dict(data=data, layout=layout)

    py.offline.iplot(fig)
    
pot_min_value=widgets.IntSlider(min=40,max=100,step=5, value=65)
pot_max_value=widgets.IntSlider(min=250,max=300,step=5, value=285)
ff_1_value=widgets.FloatSlider(min=0.1,max=1,step=0.1, value=0.6)
ff_2_value=widgets.FloatSlider(min=0.1,max=1,step=0.1, value=0.35)
    
widgets.interact(graph, pot_min=pot_min_value, pot_max=pot_max_value, ff_1=ff_1_value, ff_2=ff_2_value)

<function __main__.graph>

**Uso de la Sigmoide de Potencia**
* Supongamos un ciclista con una potencia máxima para la simulación de 275w y una potencia mínima de 70w. Podemos jugar con los factores de forma de la sigmoide para variar la forma en la que inicializamos la potencia
* El factor_forma_1 (ff_1) nos sirve para desplazar hacia las pendientes positivas el momento en que aumenta la potencia del ciclista. Usaremos valores grandes cuando por ejemplo, las bajadas del recorrido sean muy curveadas en las que no tiene sentido tratar de entregar potencia cuesta abajo
* El factor_forma_2 (ff_2) nos sirve aumentar la pendiente con la que pasamos de valores de potencia bajos a altos. Usaremos valores elevados en perfiles muy llanos, puesto que la mayor parte del tiempo la pasaremos en torno al 0%. En etapas de montaña, sería un 'suicidio' deportivo usar un ff_2 alto, pues nos haría pasar muchas horas en valores máximos de potencia que no es posible aguantar
* En general estos 4 valores, junto con el resultado de la simulación que se explica más adelante, deben ser usados de forma iterativa hasta obtener unos valores que aporten confianza al entrenador y al triatleta

>Ver el Notebook: ../Machine Learning/Warm-Start.ipynb


Ya tenemos todos los componentes para pasar a la simulación
>Ver el Notebook: ../Simulador/Simulacion.ipynb

**A continuación mostramos 2 ejemplos de uso del Simulador**

* Simulación de una prueba ya realizada. En este caso es el _Half-Ironman_ de Marbella celebrado el 29-Abril
* El segundo ejemplo, genera el plan de competición para el Ecotrimad, un triatlon distancia _Short (37kms)_ en el que participaré el próximo 16-Junio. El archivo orígen con las altimetrías ha sido descargado de Wikiloc

En ambos casos se simula una potencia media objetivo de 230w y se añade como restricción no estar más de 4.200 segundos por encima de 240w _(Dato basado en el histórico de entrenamiento personal)_

**Marbella**
------------
Marbella se trata de un recorrido de perfil montañoso, con menos metros en torno al 0% que en otro tipo de carreras, por lo que generalmente se compite 'a tope' o se está descendiendo
>Se simula con Potencia_minima = 150, Potencia_maxima = 290, factor_forma_1 = 0.7 y factor_forma_2 = 0.2

![alt text](img/Marbella_Hist.PNG "Histograma Marbella")

Vemos un Histograma con mayoría de tramos cercanos al 0%, pero con muchos valores de pendientes altas y bajas

![alt text](img/Marbella_Sig.PNG "Potencia entrada a Simulación")

Usamos una Sigmoide más _tumbada_ para no incurrir en altas potencias en los valores positivos

![alt text](img/Marbella_Sim.PNG "Fin de Simulación")

Tras la primera Simulación, obtenemos los 50 casos favorables que buscamos en 74 intentos

![alt text](img/Marbella_Scatter.PNG "Scatter Tiempo vs Potencia")

Como es lógico, vemos que a mayor potencia menor tiempo

![alt text](img/Marbella_Mejor.PNG "Hist Potencia del mejor resultado")

Este Histograma muestra la distribución de potencia del mejor resultado que arroja un tiempo de 11.460 segundos

![alt text](img/Marbella_Scatter_Pend_Pot.PNG "Scatter Pendiente vs Potencia")

Vemos las Potencias propuestas en función de la pendiente

![alt text](img/Marbella_Scatter_Viento_Pot.PNG "Scatter Viento vs Pendiente")

Si nos quedamos con un rango 'estrecho', se observa que la diferencia está provocada por el hecho de que el viento sea favorable o en contra. La Potencia está coloreada de forma que los valores claros son los de mayor Potencia

![alt text](img/Marbella_Sim_2.PNG "Fin de Simulación Segunda Ronda")

Tras la segunda Simulación, ponemos como restricción obtener 15 resultados mejores que el anterior obtenido que es usado para inicializar la Simulación. En este caso hemos obtenido un resuldado de **11.436 segundos** que apenas mejora en 24 segundos el resultado anterior

>Para mayor detalle del resultado consultar el libro de Tableau **_Resultado Simulacion GPX Standard_**


**Ecotrimad**
-------------

Al igual que Marbella se trata de un recorrido de perfil montañoso, con pocos metros en torno al 0%, por lo que también se compite 'a tope' o se está descendiendo. En este caso, intentaremos generar una estrategia que permita descansar algo más las piernas para poder correr mejor después
>Se simula con Potencia_minima = 65, Potencia_maxima = 300, factor_forma_1 = 0.5 y factor_forma_2 = 0.2

![alt text](img/Eco_Hist.PNG "Histograma Marbella")

Vemos un Histograma con más tramos cercanos al 0% y con una zona de bajadas en torno al -3% 

![alt text](img/Eco_Sig.PNG "Potencia entrada a Simulación")

Usamos una Sigmoide algo menos _tumbada_ que en Marbella, ya que tenemos mayor rango de Potencias a recorrer

![alt text](img/Eco_Sim.PNG "Fin de Simulación")

Tras la primera Simulación, obtenemos los 50 casos favorables con sólo 3 fallos. Hemos necesitado menos intentos porque las simulaciones han sido en menor medida rechazadas por incumplir la restricción explicada al inicio

![alt text](img/Eco_Scatter.PNG "Scatter Tiempo vs Potencia")

Al igual que en Marbella, vemos que a mayor potencia menor tiempo

![alt text](img/Eco_Mejor.PNG "Hist Potencia del mejor resultado")

Este Histograma muestra la distribución de potencia del mejor resultado que arroja un tiempo de 5.080 segundos

![alt text](img/Eco_Scatter_Pend_Pot.PNG "Scatter Pendiente vs Potencia")

Vemos las Potencias propuestas en función de la pendiente

![alt text](img/Eco_Scatter_Viento_Pot.PNG "Scatter Viento vs Pendiente")

Nuevamente observamos los tramos cercanos al 0% y observamos de nuevo una propuesta de mayores potencias cuando el viento es favorable

![alt text](img/Eco_Sim2.PNG "Fin de Simulación Segunda Ronda")

Realizamos una segunda pasada al Simulador y en este caso hemos obtenido un resuldado de **5.062 segundos** que mejora en 18 segundos el resultado anterior

>Para mayor detalle del resultado consultar el libro de Tableau **_Resultado Simulacion GPX Competicion Personal_**

A partir de estos primeros resultados podría repetirse tantas veces como se quisiera el proceso e ir refinando el resultado
